# Corretor Ortográfico

Aqui vamos fazer a construção de um simples corretor ortográfico que tem como intuito a correção de palavras digitadas de forma errônea. Vamos abordar alguns conceitos dentro do NLP, mas grande parte dessa construção será via algoritmos, usando slices de lista e laços de repetição em sua maior parte. Os dados usados podem ser pegos no github e podem ser usados outros dados, o que foi utilizado foi pego do site da Alura e foi e está sendo adicionado mais dados, para deixar a base mais robusta, não contendo apenas textos técnicos de tecnologia. A base de teste também vem sendo alterada. 

<center><img width='45%' src='https://cdn.pixabay.com/photo/2020/09/14/08/31/letters-5570355_960_720.jpg'>
 
 


Como dito acima, esse corretor pode ser replicado para novos tipos de dados e otimizado também, aqui vamos fazer duas variações, mas devido ao conjunto de teste que vamos usar ele vai tender a ficar com a variação mais simples. Com isso, uma mudança de dados teste pode usar a sua outra alteração. O intuito deste corretor é ele enquadrar nos dados, que remete a situações simples de digitação. 

## Importando os arquivos

In [ ]:
with open('Base_de_dados.txt', 'r') as f:
  texto_base = f.read()

In [3]:
# aqui vamos apenas ver os 100 primeiros caracteres
print(texto_base[:100]) 




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar u


## Trabalhando o Dataset

Antes de iniciar os processos de modificações precisamos aplicar a tokenização, isso quer dizer que vamos pegar os nossos dados e transformar em palavras e não em caracteres, a palavra `legal` será contabilizada de tamanho 1 e não 5. Isso é importante para criação do nosso corretor e para avaliação da sua performance. Para esse processo vamos fazer uso da biblioteca NLTK, que já é pronta para isso, caso queira se aprofundar pode se encontrar sua documentação aqui: https://www.nltk.org/

Aqui fazemos uso do `punkt`, uma rápida explicação sobre esse módulo: O Punkt é feito para aprender parâmetros de um corpus de forma não supervisionada e relacionados ao domínio de destino, como uma lista de abreviações, acrônimos, etc.

fonte: https://www.askpython.com/python-modules/nltk-punkt

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# criando uma função para separar as plavras e suas pontuações ("?,.-"")
def palavras(tokens):
  palavras = []
  for token in tokens:
    if token.isalpha(): # aqui estou dizendo que é  alfabetico
      palavras.append(token)
  return palavras

In [6]:
# aplicando a função em nossos dados 
tokens = nltk.tokenize.word_tokenize(texto_base)
palavras = palavras(tokens)

print(f'O número total de palavras é {len(palavras)}') 

O número total de palavras é 403031


In [8]:
# verificando a tokenização
print(palavras[:10])

['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


Aqui temos que aplicar uma normalização pelo fato de termos as mesmas palavras mas com formatos diferentes, um exemplo: Legal e legal. Para isso vamos aplicar uma normalização geral em todo nosso dataset.

In [13]:
def normalizacao(palavras):
  normalizacao = []
  for palavra in palavras:
    normalizacao.append(palavra.lower()) # aqui apenas estou transformando tudo em minúscula 
  return normalizacao

In [14]:
# verificando a normalização
palavras_normalizadas = normalizacao(palavras)
print(palavras_normalizadas[:10])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


Agora vamos apenas aplicar o `set` assim vamos saber o número de palavras únicas dentro da nossa base de dados.

In [15]:
len(set(palavras_normalizadas))

18464

## Resolvendo os problemas de digitação

Aqui vamos abordar 3 problemas de digitação e considerando uma distância, o termo distância eu explico mais tarde quando abordar um problema com mais de uma distância. Os quatro problemas que vamos trabalhar será os mais comuns que temos ao digitar uma palavra ou frase:

* Esquecer de digitar uma letra;
* Digitar uma letra em vez de outra;
* Digitar uma palavra a mais;
* Trocando a posição das letras;

Para isso precisamos "fatiar" nossas palavras, para isso vamos usar o slice da lista, pois como queremos acrescentar, substituir e remover caracteres de palavras, primeiramente necessitamos fatiá-las. No caso esses cortes são feitos a esquerda e a direita, por exemplo: alegal, leagal e assim por diante, em código ficará mais fácil a visualização. Vamos lá.

In [65]:
exemplo1 = 'lgal'
exemplo2 = 'leagal'
exemplo3 = 'lagal'
exemplo4 = 'legla' 
exemplo5= 'leaigal'

In [35]:
# inserindo caracteres
def inserir(slice):
  palavras_novas = []
  letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèíîìóõòúûùç' # tudo em minusculo, pois já normalizamos
  for e, d in slice: # as letras aqui servem apenas para indicar esquerda e direita
    for letra in letras:
      palavras_novas.append(e + letra + d) # aqui estamos fazendo o ascrescimo da palavra, como explicado acima
  return palavras_novas


In [36]:
# deletando caracteres
def delete(slice):
  palavras_novas = []
  for e, d in slice:
    palavras_novas.append(e + d[1:]) # aqui não pegamos a palavra de index 0, excluindo ela
  return palavras_novas

In [41]:
# trocando caracter
def trocador(slice):
  palavras_novas = []
  letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèíîìóõòúûùç'
  for e, d in slice:
    for letra in letras:
      palavras_novas.append(e + letra + d[1:]) # aqui estamos pegano o caracter de index 0 e substituindo ele por um outro char
  return palavras_novas

In [46]:
# inversor de caracter
def inversor(slice):
  palavras_novas = []
  for e, d in slice:
    if len(d) > 1: # o lado direito da palavra deve ter pelo menos 2 char, pois caso contrario não é possivel fazer a troca
      palavras_novas.append(e + d[1] + d[0] + d[2:]) # inversão das palavras, caso queira enteder o processo pode pegar uma palavra e ir fazendoos slices, para a compreensão
  return palavras_novas

Aqui criei a função geradora de palavra para ir testando as funções e depois aplicar no avaliador, vamos usar ela no final mas deixarei ele também aqui para que se possa ir testando palavras e suas correções.

In [47]:
def gerador(palavra):
  slice = []
  for i in range(len(palavra) + 1): # pegar o último item
    slice.append((palavra[:i], palavra[i:])) # aqui estamos pegando a palavra "picotada"
  palavras_geradas = inserir(slice) 
  palavras_geradas += delete(slice)
  palavras_geradas += trocador(slice)
  palavras_geradas += inversor(slice)
  return palavras_geradas

palavras_g = gerador(exemplo4) # aqui você pode ir alterando os exemplos para testar cada função
print(palavras_g)

['alegla', 'blegla', 'clegla', 'dlegla', 'elegla', 'flegla', 'glegla', 'hlegla', 'ilegla', 'jlegla', 'klegla', 'llegla', 'mlegla', 'nlegla', 'olegla', 'plegla', 'qlegla', 'rlegla', 'slegla', 'tlegla', 'ulegla', 'vlegla', 'wlegla', 'xlegla', 'ylegla', 'zlegla', 'álegla', 'âlegla', 'àlegla', 'ãlegla', 'élegla', 'êlegla', 'èlegla', 'ílegla', 'îlegla', 'ìlegla', 'ólegla', 'õlegla', 'òlegla', 'úlegla', 'ûlegla', 'ùlegla', 'çlegla', 'laegla', 'lbegla', 'lcegla', 'ldegla', 'leegla', 'lfegla', 'lgegla', 'lhegla', 'liegla', 'ljegla', 'lkegla', 'llegla', 'lmegla', 'lnegla', 'loegla', 'lpegla', 'lqegla', 'lregla', 'lsegla', 'ltegla', 'luegla', 'lvegla', 'lwegla', 'lxegla', 'lyegla', 'lzegla', 'láegla', 'lâegla', 'làegla', 'lãegla', 'léegla', 'lêegla', 'lèegla', 'líegla', 'lîegla', 'lìegla', 'lóegla', 'lõegla', 'lòegla', 'lúegla', 'lûegla', 'lùegla', 'lçegla', 'leagla', 'lebgla', 'lecgla', 'ledgla', 'leegla', 'lefgla', 'leggla', 'lehgla', 'leigla', 'lejgla', 'lekgla', 'lelgla', 'lemgla', 'lengla',

In [48]:
# verificando se a palavra foi corrigida
teste = 'legal'
teste in palavras_g

True

## Criando Avaliador

Aqui vamos criar nosso avaliador para que possamos saber como está a eficácia do nosso corretor e ver as possíveis melhoras.

In [49]:
# criando a função probabilidade
frequencia = nltk.FreqDist(palavras_normalizadas) # a distribuição de frequências das palavras
def prob(palavra_gerada):
  return frequencia[palavra_gerada] / len(palavras_normalizadas)

In [50]:
# criando o corretor
def corretor(palavra):
  palavras_geradas = gerador(palavra)
  palavra_correta = max(palavras_geradas, key=prob) # aqui me retorna a chance de ser a palavra correta, no caso a palavra que tiver a maior probabilidade de ser a certa
  return palavra_correta

In [51]:
def teste(dados_teste):
  teste = []
  f = open(dados_teste, 'r')
  for i in f:
    certa, errada = i.split() # aqui vamos separar por espaço as palavras
    teste.append((certa, errada)) # criando uma tupla com as palavras
  f.close()
  return teste

In [52]:
teste = teste('Teste.txt')
print(teste[:10]) # visualizando nossa lista teste

[('podemos', 'pyodemos'), ('esse', 'esje'), ('já', 'jrá'), ('nosso', 'nossov'), ('são', 'sãêo'), ('dos', 'dosa'), ('muito', 'muifo'), ('imagem', 'iômagem'), ('sua', 'ósua'), ('também', 'tambéùm')]


In [68]:
def avaliador (teste, base_palavras):
  total_palavras = len(teste)
  acertos = 0
  desconhece = 0 # aqui são palavras que não estão na nossa base de dados
  for certa, errada in teste:
    corrigida = corretor(errada) 
    desconhece += (certa not in base_palavras) # a palavra correta da nossa base de dados não está dentro da nossa base de dados
    if corrigida == certa:
      acertos += 1   
  taxa_de_acertos = round(acertos * 100 / total_palavras, 2)
  taxa_de_desconhecidas = round(desconhece * 100 / total_palavras, 2)
  print(f'Taxa de acerto {taxa_de_acertos}% de {total_palavras} palavras, e uma porcentagem de {taxa_de_desconhecidas}% palavras desconhecidas')


In [60]:
base_palavras = set(palavras_normalizadas)
avaliador(teste, base_palavras)

Taxa de acerto 76.34% de 186 palavras, e uma porcentagem de 6.99% palavras desconhecidas


Aqui finalizamos o nosso corretor ortográfico, ele corrige palavras com até uma distância, podemos alterar ele para que corrija por mais de uma distância, o que vamos fazer logo abaixo, porém nossa base de dados é voltada para erros de uma distância, o que é mais comum de ocorrer. Junto a essa nova função irei explicar um pouco sobre distância, vamos lá!

## Corretor com mais de uma distância

A distânica esta relacioanada com a quantidade de oprações que devemos fazer para efetuar a coreção da palavra. Veja o exemplo abaixo:
* leoagal -> 2 operações;
* luggal -> 3 operações;

In [61]:
def gerador_distancia(palavras):
  palavras_novas = []
  for palavra in palavras:
    palavras_novas += gerador(palavra) # para cada palavra que passa no gerador eu aplico novamente o gerador
  return palavras_novas

In [62]:
def corretor_atualizado(palavra):
  palavras_geradas = gerador(palavra)
  palavras_distancia = gerador_distancia(palavras_geradas)
  total_palavras = set(palavras_geradas + palavras_distancia)
  possiveis_palavras = [palavra]
  for palavra in total_palavras:
    if palavra in base_palavras:
      possiveis_palavras.append(palavra)
  correta = max(possiveis_palavras, key=prob)
  return correta

In [66]:
corretor_atualizado(exemplo5)

'legal'

Aqui vamos fazer uma simples alteração no nosso avaliador para ele apresentar a taxa de acerto do nosso corretor.

In [69]:
def avaliador_novo (teste, base_palavras):
  total_palavras = len(teste)
  acertos = 0
  desconhece = 0 # aqui são palavras que não estão na nossa base de dados
  for certa, errada in teste:
    corrigida = corretor_atualizado(errada) 
    desconhece += (certa not in base_palavras) # a palavra correta da nossa base de dados não está dentro da nossa base de dados
    if corrigida == certa:
      acertos += 1   
  taxa_de_acertos = round(acertos * 100 / total_palavras, 2)
  taxa_de_desconhecidas = round(desconhece * 100 / total_palavras, 2)
  print(f'Taxa de acerto {taxa_de_acertos}% de {total_palavras} palavras, e uma porcentagem de {taxa_de_desconhecidas}% palavras desconhecidas')


In [70]:
avaliador_novo(teste, base_palavras)

Taxa de acerto 55.38% de 186 palavras, e uma porcentagem de 6.99% palavras desconhecidas


Aqui podemos observar que nossa taxa de acerto caiu, isso se deve ao fato de o novo gerador gerar novas palavras que se encaixam mais em palavras que se repetem mais no texto e como estamos usando a probabilidade de ser a palavra sendo. Porém, os dados teste se enquadram melhor no corretor mais simples, por isso este novo não se sai muito bem. Entretanto, podemos fazer alterações para combinar os dois, assim gerando um corretor otimizado.
